<a href="https://colab.research.google.com/github/RDGopal/IB9AU-2026/blob/main/MLM4_Diffusion_1D_Teaching_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 Diffusion Models in 1D — From Data to Gaussian and Back

This notebook builds a **1D diffusion model** end‑to‑end on a **multi‑modal** data distribution.  
We will:
1) Create a **quad‑modal** data distribution and sample from it.  
2) Implement the **forward diffusion** process (adding Gaussian noise step by step) and visualize how the data **becomes Gaussian**.  
3) **Collect the right training tuples** from the forward process.  
4) Train a tiny neural net to **predict noise** $\epsilon$-prediction.  
5) Use the network to **run the reverse diffusion** and regenerate samples from pure noise.

**References for theory (recommended reading):**
- Ho, Jain, Abbeel (2020), *Denoising Diffusion Probabilistic Models* (DDPM), arXiv:2006.11239.
- Sohl‑Dickstein et al. (2015), *Deep Unsupervised Learning using Nonequilibrium Thermodynamics*.
- Nichol & Dhariwal (2021), *Improved DDPM* (cosine schedule).
- Song et al. (2020), *Score‑based Generative Modeling through SDEs*.

> This notebook uses **only 1D data** so everything trains very quickly on CPU.


##Setup

We rely on PyTorch for a tiny MLP and Matplotlib for plots.

In [ ]:
# If running on Colab, uncomment the next line to ensure torch is installed/updated.
# !pip -q install torch --upgrade

import math, random, numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)


## Build a **quad‑modal** 1D data distribution

We will define a mixture of four Gaussians with different means, scales, and weights.  Then we draw samples and visualize the histogram.


In [ ]:
# Mixture of Gaussians (quad-modal)
weights = np.array([0.20, 0.30, 0.10, 0.40])  # must sum to 1
means   = np.array([-6.0, -2.0,  2.0,  6.0])
stds    = np.array([ 0.5,  0.3,  0.4,  0.6])

assert np.isclose(weights.sum(), 1.0)

N = 50000  # number of data points
comps = np.random.choice(len(weights), size=N, p=weights)
x0 = np.random.normal(loc=means[comps], scale=stds[comps]).astype(np.float32)

def plot_hist(data, title, bins=200, range_=(-10,10)):
    plt.figure()
    plt.hist(data, bins=bins, range=range_, density=True)
    plt.title(title)
    plt.xlabel("x")
    plt.ylabel("density")
    plt.show()

plot_hist(x0, "Original data distribution (quad-modal)")


## Forward diffusion (DDPM style)


We define a variance schedule
$\{\beta_1,...,\beta_T\}$ and the forward Markov chain:

$$
q(x_t \mid x_{t-1}) = \mathcal{N}\!\big(\sqrt{1-\beta_t}\,x_{t-1},\, \beta_t\big)$$

From this it follows that for any $t$:

$$
x_t = \sqrt{\bar\alpha_t}\,x_0 + \sqrt{1-\bar\alpha_t}\,\epsilon,\quad \epsilon\sim\mathcal{N}(0,1) $$

where
$$
\alpha_t = 1-\beta_t  \  \& \  
 \bar\alpha_t=\prod_{s=1}^t \alpha_s
$$

We'll choose a small number of steps $T$ (e.g., 300) and a simple **linear** \($\beta_t\$) schedule for clarity.


In [ ]:
# Diffusion schedule
T = 300
beta_start, beta_end = 1e-4, 0.02  # simple linear schedule
betas = torch.linspace(beta_start, beta_end, T)
alphas = 1.0 - betas
alpha_bars = torch.cumprod(alphas, dim=0)

def q_sample(x0, t, noise=None):
    """Sample x_t given x0 in closed form:
    x_t = sqrt(alpha_bar_t) * x0 + sqrt(1 - alpha_bar_t) * epsilon
    """
    if noise is None:
        noise = torch.randn_like(x0)
    sqrt_ab = torch.sqrt(alpha_bars[t]).to(x0.device)
    sqrt_one_minus_ab = torch.sqrt(1.0 - alpha_bars[t]).to(x0.device)
    return sqrt_ab * x0 + sqrt_one_minus_ab * noise, noise


In [ ]:
torch.set_printoptions(sci_mode=False)
print(betas)

### Visualize the forward process

We'll show the histogram of $x_t$ at several time steps to see the distribution **wash out** towards a standard Gaussian.


In [ ]:
# Visualize forward diffusion histograms at selected steps
xs = torch.from_numpy(x0).unsqueeze(1)  # shape [N,1]
steps = [0, 10, 50, 100, 150, 200, 250, 299]  # (0 indexes t=1 as 0 in code; T-1 is the last index)

for s in steps:
    if s == 0:
        xt = xs.clone()
        data = xt.squeeze(1).numpy()
        plot_hist(data, f"x_0 (t=0)")
    else:
        with torch.no_grad():
            xt, _ = q_sample(xs, s-1)
            data = xt.squeeze(1).numpy()
            plot_hist(data, f"x_{s} (forward diffusion step {s})")


## What data do we need for **reverse** diffusion?

In DDPM training, the model learns to predict the injected noise $\epsilon$.  
To train it, we **collect tuples** $(x_t, t, \epsilon)$ generated from:

$$ x_t = \sqrt{\bar\alpha_t}\,x_0 + \sqrt{1-\bar\alpha_t}\,\epsilon,\ \ \epsilon\sim\mathcal{N}(0,1) $$

We will synthesize a dataset of such tuples from our 1D data $x_0$.


In [ ]:
# Build a synthetic training set of (x_t, t, epsilon)
N_train = 20000
x0_train = torch.from_numpy(np.random.choice(x0, size=N_train, replace=True)).float().unsqueeze(1)
t_train = torch.randint(low=0, high=T, size=(N_train,))
eps = torch.randn_like(x0_train)

# closed-form sample of x_t and record epsilon used
sqrt_ab = torch.sqrt(alpha_bars[t_train]).unsqueeze(1)
sqrt_one_minus_ab = torch.sqrt(1.0 - alpha_bars[t_train]).unsqueeze(1)
x_t_train = sqrt_ab * x0_train + sqrt_one_minus_ab * eps

# peek at a few samples
for i in range(3):
    print("t={}, x_t={:.3f}, epsilon={:.3f}".format(int(t_train[i]), float(x_t_train[i]), float(eps[i])))


Let's write out the training data that will be used for training the reverse diffusion process into a data frame.

In [ ]:
import pandas as pd

# Convert tensors to numpy arrays and remove the extra dimension
x_t_np = x_t_train.squeeze(1).numpy()
t_np = t_train.numpy()
eps_np = eps.squeeze(1).numpy()

# Create a dictionary with the data
training_data = {
    't': t_np,
    'x_t': x_t_np,
    'epsilon': eps_np
}

# Create the DataFrame
df_train = pd.DataFrame(training_data)

# Display the first few rows of the DataFrame
display(df_train.head())

## A tiny network to predict $\epsilon_\theta(x_t, t)$

We will train a very small MLP on 1D inputs to predict the noise $\epsilon$ from $(x_t, t)$.  
We embed $t$ with a **sinusoidal embedding** (like positional encodings).
The training loss is $(\mathbb{E}\|\epsilon - \epsilon_\theta(x_t,t)\|^2)$.


### Why sinusoidal embedding for time $t$?

Instead of feeding the raw timestep
$t$ (an integer like 137) into the network, we map $t$ to a vector made of sines and cosines at many frequencies:

$ PE(t,2i) = sin(t/10000^{2i/d}) $,
$ PE(t,2i+1) = cos(t/10000^{2i/d}) $,
$(i=0,…,d/2-1)$




This idea comes from the Transformer's positional encoding: each dimension is a sinusoid with a different wavelength, which helps the model infer relative offsets (how far apart two positions are) and even extrapolate to positions it never saw, because the pattern is analytic (not looked up).

In diffusion models we need the network to “know” the noise level / time step it is denoising at. So we encode $t$ with these $sin/cos$ features and feed that vector into the MLP. we turn a single number $t$ into a rich, multi-frequency signal the network can use to condition its predictions.


### Neural network that predicts the noise ($\epsilon$).

1.  **`TimeEmbedding` Class**:
    *   This class creates a sinusoidal embedding for the time step $t$. It takes an integer time step and transforms it into a vector of sine and cosine values at different frequencies. This helps the network understand the notion of time and how far along the diffusion process it is.

2.  **`EpsilonNet` Class**:
    *   This is the main neural network model.
    *   It uses the `TimeEmbedding` class to process the time step $t$.
    *   The network itself is a simple Multi-Layer Perceptron (MLP) with two hidden layers and SiLU activation functions.
    *   The input to the MLP is the concatenation of the current noisy data point $x_t$ and the time embedding vector.
    *   The output of the MLP is a single value, which is the predicted noise $\epsilon$.

3.  **Model Initialization and Optimizer**:
    *   `model = EpsilonNet()`: Creates an instance of the `EpsilonNet`.
    *   `opt = optim.AdamW(model.parameters(), lr=2e-3)`: Sets up the AdamW optimizer to train the model, with a learning rate of 0.002.

4.  **`batch_iter` Function**:
    *   This function is a simple data loader. It randomly samples batches of `x_t`, `t`, and `eps` from the training data `x_t_train`, `t_train`, and `eps`.

5.  **Training Loop**:
    *   The code runs a training loop for 2000 steps.
    *   In each step:
        *   `batch_iter()`: Gets a batch of training data.
        *   `model(x_b, t_b)`: Feeds the noisy data and time steps into the model to get the predicted noise.
        *   `F.mse_loss(pred, e_b)`: Calculates the Mean Squared Error (MSE) between the predicted noise and the actual noise. This is the loss function the model tries to minimize.
        *   `opt.zero_grad(); loss.backward(); opt.step()`: Performs a standard PyTorch training step: clears previous gradients, computes gradients of the loss with respect to model parameters, and updates the model parameters using the optimizer.
        *   The loss is printed every 200 steps to monitor training progress.

6.  **Plotting Loss**:
    *   After training, the code plots the recorded loss values to visualize how the training progressed over time.

In essence, this code defines a neural network that learns to predict the noise added at any given time step of the forward diffusion process, which is a crucial component for the reverse diffusion (sampling) process.


The **SiLU (Sigmoid Linear Unit)** activation function is a type of activation function used in neural networks. It's defined as:

$ SiLU(x) = x * \sigma(x) $

where $\sigma(x)$ is the sigmoid function.

Here's why it's used:

1. Smoothness: Unlike the ReLU function, SiLU is smooth everywhere, which can help with optimization during training.
2. Non-monotonic: It's not monotonic, which allows it to handle more complex patterns in the data.
3. Performance: It has been shown to perform well in various neural network architectures, sometimes outperforming other common activation functions like ReLU and Swish.

In simple terms, it's a function that introduces non-linearity into the network, allowing it to learn complex relationships in the data.

In [ ]:
# Sinusoidal time embedding for 1D
class TimeEmbedding(nn.Module):
    def __init__(self, dim=32, max_period=10000):
        super().__init__()
        self.dim = dim
        self.max_period = max_period
    def forward(self, t):  # t: [B] integer 0..T-1
        device = t.device
        half = self.dim // 2
        freqs = torch.exp(-math.log(self.max_period) * torch.arange(half, device=device) / half)
        args = t.float().unsqueeze(1) * freqs.unsqueeze(0)
        emb = torch.cat([torch.sin(args), torch.cos(args)], dim=1)
        if self.dim % 2 == 1:
            emb = torch.cat([emb, torch.zeros_like(emb[:, :1])], dim=1)
        return emb

class EpsilonNet(nn.Module):
    def __init__(self, t_dim=32, hidden=64):
        super().__init__()
        self.tembed = TimeEmbedding(dim=t_dim)
        self.net = nn.Sequential(
            nn.Linear(1 + t_dim, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, 1),
        )
    def forward(self, x_t, t):
        te = self.tembed(t)
        h = torch.cat([x_t, te], dim=1)
        return self.net(h)

model = EpsilonNet()
opt = optim.AdamW(model.parameters(), lr=2e-3)

# simple training loop
def batch_iter(bs=512):
    n = x_t_train.shape[0]
    idx = torch.randint(0, n, (bs,))
    return x_t_train[idx], t_train[idx], eps[idx]

losses = []
for step in range(2000):  # quick train; increase for slightly better results
    x_b, t_b, e_b = batch_iter()
    pred = model(x_b, t_b)
    loss = F.mse_loss(pred, e_b)
    opt.zero_grad(); loss.backward(); opt.step()
    if (step+1) % 200 == 0:
        losses.append(float(loss.detach().cpu()))
        print("step {}: loss {:.6f}".format(step+1, loss.item()))

# plot training loss
plt.figure()
plt.plot(losses)
plt.title("Training loss (every 200 steps)")
plt.xlabel("checkpoint")
plt.ylabel("MSE")
plt.show()


##  Reverse diffusion (sampling) with the learned $\epsilon_\theta$

In DDPM with $\epsilon$ prediction, the mean of the reverse transition is:

$$
\mu_\theta(x_t,t) = \frac{1}{\sqrt{\alpha_t}}\!\left(x_t - \frac{1-\alpha_t}{\sqrt{1-\bar\alpha_t}}\ \epsilon_\theta(x_t,t)\right)
$$

We'll step from

$x_T\sim\mathcal{N}(0,1)$ **down to** $x_0$
 using this mean and the known variance.


In [ ]:
# One reverse step using predicted epsilon
def p_sample_step(x_t, t_idx):
    t = torch.full((x_t.shape[0],), t_idx, dtype=torch.long)
    eps_theta = model(x_t, t)
    a_t = (1.0 - betas[t_idx]).item()
    ab_t = alpha_bars[t_idx].item()
    coef = (1.0 - a_t) / math.sqrt(1.0 - ab_t)
    mean = (1.0 / math.sqrt(a_t)) * (x_t - coef * eps_theta)
    if t_idx > 0:
        var = betas[t_idx].item()
        noise = torch.randn_like(x_t)
        x_prev = mean + math.sqrt(var) * noise
    else:
        x_prev = mean  # at t=0, set variance to 0 for final sample
    return x_prev

# Run the reverse process
with torch.no_grad():
    n_samp = 20000
    xT = torch.randn(n_samp, 1)  # start from standard normal
    x = xT
    for t_idx in reversed(range(T)):
        x = p_sample_step(x, t_idx)
    x_gen = x.squeeze(1).cpu().numpy()

plot_hist(x_gen, "Generated samples after reverse diffusion")


## Compare original vs. generated

Let's overlay histograms (separate plots) and then visually compare the shape of the learned distribution to the original quad‑modal data.


In [ ]:
plot_hist(x0, "Original data distribution (quad-modal) — reference")
plot_hist(x_gen, "Generated distribution — reverse diffusion")


## Recap — What we *collected* during forward diffusion

To enable reverse diffusion (learning), we created tuples:
- $x_0 \sim q_{\text{data}}$ (our mixture‑of‑Gaussians sample)
- Sample $t \sim \{0,\dots,T-1\}$
- Sample $\epsilon \sim \mathcal{N}(0,1)$
- Compute $x_t = \sqrt{\bar\alpha_t}\,x_0 + \sqrt{1-\bar\alpha_t}\,\epsilon$

**Training data:** $(x_t, t, \epsilon)$.  
**Model objective:** minimize $\mathbb{E}\|\epsilon - \epsilon_\theta(x_t,t)\|^2$.  
**Sampling:** start from $x_T \sim \mathcal{N}(0,1)$, apply reverse steps using $\mu_\theta(x_t,t)$.


---

### Appendix: Notes & tips
- Increase **training steps** if the generated histogram looks too smooth or too noisy. Even in 1D, a little more training helps.

**Further reading (primary sources):**
- DDPM: https://arxiv.org/abs/2006.11239  
- Sohl‑Dickstein et al. (2015): https://proceedings.mlr.press/v37/sohl-dickstein15.html  
- Improved DDPM (cosine schedule): https://proceedings.mlr.press/v139/nichol21a/nichol21a.pdf  
- Score‑based SDEs: https://arxiv.org/abs/2011.13456
